# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def df_nhl(): 
    nhl=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nhl = nhl[nhl["year"]==2018]
    nhl.drop(nhl[nhl['GP'].str.contains('[a-zA-Z]+')].index, inplace=True)
    nhl = nhl.reset_index()
    nhl = nhl.loc[:, ["team","L", "W"]]
    nhl["team"] =nhl.team.replace(to_replace="\s*\**\s*",value="",regex=True)
    nhl["L"] = nhl["L"].astype(float)
    nhl["W"] = nhl["W"].astype(float)

    def ratio(x):
        result = x["W"]/(x["W"]+x["L"])
        return result

    nhl["winrate"]  = nhl.apply(ratio, axis=1)
    del nhl["L"]
    del nhl["W"]
    new_df = nhl["team"].str.extract("(?P<City>^[\w\.]+)(?P<team>[A-Z]+[a-z]+$)")
    nhl["city"] = new_df["City"]
    nhl["team"] = new_df["team"]
    nhl = nhl.set_index("team")

    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities= cities.loc[:, ["Metropolitan area", "Population (2016 est.)[8]", "NHL"]]
    cities["NHL"] =cities.NHL.replace(to_replace="\s*\[[^()]*\]\s*$",value="",regex=True)     
    cities.loc[cities["NHL"] == "—", "NHL"] = float("nan")
    cities.loc[cities["NHL"] == "", "NHL"] = float("nan")
    cities = cities.dropna()
    cities.rename(columns = {"NHL": "team", "Population (2016 est.)[8]" : "Poblacion"}, inplace= True)
    cities["Poblacion"] = cities["Poblacion"].astype(float)
    nhl["winrate"] = nhl["winrate"].astype(float)
    teams = cities["team"].str.extract("(?P<team1>^[\w]+)(?:[\s]*)(?P<team2>[\w]*)(?:[\s]*)(?P<team3>[\w]*$)")
    cities["team"] = teams["team1"]

    copia = pd.DataFrame()
    copia["Metropolitan area"] = cities["Metropolitan area"]
    copia["Poblacion"] = cities["Poblacion"]
    copia["team"] = teams["team2"]
    copia = copia[copia["team"] != ""]

    copia2 = pd.DataFrame()
    copia2["Metropolitan area"] = cities["Metropolitan area"]
    copia2["Poblacion"] = cities["Poblacion"]
    copia2["team"] = teams["team3"]
    copia2 = copia2[copia2["team"] != ""]

    cities = cities.set_index("team")
    copia = copia.set_index("team")
    copia2 = copia2.set_index("team")

    merge1 = pd.merge(nhl,cities,how="inner", left_index= True, right_index=True)
    merge2 = pd.merge(nhl,copia,how="inner", left_index= True, right_index=True)
    merge3 = pd.merge(nhl,copia2,how="inner", left_index= True, right_index=True)
    merge = pd.concat((merge1,merge2,merge3))
    
    merge=merge.groupby('Metropolitan area').agg({'winrate': np.mean, 'Poblacion': np.mean})
    
    return merge

def nhl_correlation(): 
    merge= df_nhl()
    merge["Poblacion"] = merge["Poblacion"].astype(int)
    population_by_region = merge["Poblacion"].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = merge["winrate"].to_list() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

ModuleNotFoundError: No module named 'pandas'

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [97]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def df_nba():
    
    nba=pd.read_csv("assets/nba.csv", skipfooter=132)
    nba = nba.loc[:, ["team", "W/L%"]]
    nba["team"] =nba.team.replace(to_replace="\**\s*\([^()]*\)\s*$",value="",regex=True)

    new_df = nba["team"].str.extract("(?P<City>^[\w]+ *[\w]*)(?:[\s]+)(?P<team>[\w]+$)")
    nba["team"] = new_df["team"]
    nba= nba.set_index("team")

    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities= cities.loc[:, ["Metropolitan area", "Population (2016 est.)[8]", "NBA"]]
    cities["NBA"] =cities.NBA.replace(to_replace="\s*\[[^()]*\]\s*$",value="",regex=True)     
    cities.loc[cities["NBA"] == "—", "NBA"] = float("nan")
    cities.loc[cities["NBA"] == "", "NBA"] = float("nan")
    cities = cities.dropna()
    cities.rename(columns = {"NBA": "team", "Population (2016 est.)[8]" : "Poblacion"}, inplace= True)
    cities["Poblacion"] = cities["Poblacion"].astype(float)
    nba["W/L%"] = nba["W/L%"].astype(float)
    teams = cities["team"].str.extract("(?P<team1>^[\w]+)(?:[\s]*)(?P<team2>[\w]*$)")
    cities["team"] = teams["team1"]
    copia = pd.DataFrame()
    copia["Metropolitan area"] = cities["Metropolitan area"]
    copia["Poblacion"] = cities["Poblacion"]
    copia["team"] = teams["team2"]
    copia = copia[copia["team"] != ""]
    cities = cities.set_index("team")
    copia = copia.set_index("team")


    merge1 = pd.merge(nba,cities,how="inner", left_index= True, right_index=True)
    merge2 = pd.merge(nba,copia,how="inner", left_index= True, right_index=True)
    merge = pd.concat((merge1,merge2))
    
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Poblacion': np.nanmean})
    
    return merge

def nba_correlation():
    merge = df_nba()
    population_by_region = merge["Poblacion"] # pass in metropolitan area population from cities
    win_loss_by_region = merge["W/L%"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

-0.17636350642182938

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [98]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def df_mlb(): 
    mlb=pd.read_csv("assets/mlb.csv")
    mlb=mlb[mlb["year"] == 2018]
    
    mlb= mlb.loc[:, ["team", "W", "L"]]
    
    mlb["L"] = mlb["L"].astype(float)
    mlb["W"] = mlb["W"].astype(float)
    def ratio(x):
        result = x["W"]/(x["W"]+x["L"])
        return result
    mlb["W-L%"] = mlb.apply(ratio, axis=1)
    mlb["W-L%"] = mlb["W-L%"].astype(float)
    del mlb["W"]
    del mlb["L"]
    mlb.at[0,"team"] = "Boston SoxR"
    mlb.at[8,"team"] = "Chicago SoxW"
    new_df = mlb["team"].str.extract("(?P<City>^[\w.]+ *[\w]*)(?:[\s]+)(?P<team>[\w]+$)")
   
    mlb["team"] = new_df["team"]
    
    mlb= mlb.set_index("team")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities= cities.loc[:, ["Metropolitan area", "Population (2016 est.)[8]", "MLB"]]
    cities["MLB"] =cities.MLB.replace(to_replace="\s*\[[^()]*\]\s*$",value="",regex=True)     
    cities.loc[cities["MLB"] == "—", "MLB"] = float("nan")
    cities.loc[cities["MLB"] == "", "MLB"] = float("nan")
    cities = cities.dropna()
    cities.rename(columns = {"MLB": "team", "Population (2016 est.)[8]" : "Poblacion"}, inplace= True)
    cities["Poblacion"] = cities["Poblacion"].astype(float)
    cities.at[3,"team"] = "Cubs SoxW"
    cities.at[7,"team"] = "SoxR"
    cities.rename(index={"Cubs White Sox" : "Cubs SoxW", })
    teams = cities["team"].str.extract("(?P<team1>^[\w]+)(?:[\s]*)(?P<team2>[\w]*)(?:[\s]*)(?P<team3>[\w]*$)")
    cities["team"] = teams["team1"]
    
    copia = pd.DataFrame()
    copia["Metropolitan area"] = cities["Metropolitan area"]
    copia["Poblacion"] = cities["Poblacion"]
    copia["team"] = teams["team2"]
    copia = copia[copia["team"] != ""]

    copia2 = pd.DataFrame()
    copia2["Metropolitan area"] = cities["Metropolitan area"]
    copia2["Poblacion"] = cities["Poblacion"]
    copia2["team"] = teams["team3"]
    copia2 = copia2[copia2["team"] != ""]

    cities = cities.set_index("team")
    copia = copia.set_index("team")
    copia2 = copia2.set_index("team")
    
    merge1 = pd.merge(mlb,cities,how="inner", left_index= True, right_index=True)
    merge2 = pd.merge(mlb,copia,how="inner", left_index= True, right_index=True)
    merge3 = pd.merge(mlb,copia2,how="inner", left_index= True, right_index=True)
    merge = pd.concat((merge1,merge2,merge3))
    merge["Poblacion"] = merge["Poblacion"].astype(int)
    final = merge.groupby("Metropolitan area").agg({"W-L%": "mean", "Poblacion": "first"})

    return final

def mlb_correlation(): 
    merge = df_mlb()
    population_by_region = merge["Poblacion"]
    win_loss_by_region = merge["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [99]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def df_nfl(): 
    # YOUR CODE HERE
    NFL=pd.read_csv("assets/nfl.csv")
    NFL=NFL[NFL["year"] == 2018]
    NFL.drop(NFL[NFL['W-L%'].str.contains('[a-zA-Z]+')].index, inplace=True)
    NFL["team"] =NFL.team.replace(to_replace="[\+]*[\*]*$",value="",regex=True)
    new_df = NFL["team"].str.extract("(?P<City>^[\w.]+ *[\w]*)(?:[\s]+)(?P<team>[\w]+$)")
    NFL= NFL.loc[:, ["team", "W", "L","T"]]
    NFL["L"] = NFL["L"].astype(float)
    NFL["W"] = NFL["W"].astype(float)
    NFL["T"] = NFL["T"].astype(float)
    def ratio(x):
        result = x["W"]/(x["W"]+x["L"])
        return result
    NFL["W-L%"] = NFL.apply(ratio, axis=1)
    NFL["W-L%"] = NFL["W-L%"].astype(float)
    del NFL["W"]
    del NFL["L"]
    del NFL["T"]
    NFL["W-L%"] = NFL["W-L%"].astype(float)
    NFL["team"] = new_df["team"]
    NFL= NFL.set_index("team")

    
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities= cities.loc[:, ["Metropolitan area", "Population (2016 est.)[8]", "NFL"]]
    cities["NFL"] =cities.NFL.replace(to_replace="\s*\[[^()]*\]\s*$",value="",regex=True)     
    cities.loc[cities["NFL"] == "—", "NFL"] = float("nan")
    cities.loc[cities["NFL"] == "", "NFL"] = float("nan")
    cities = cities.dropna()
    cities.rename(columns = {"NFL": "team", "Population (2016 est.)[8]" : "Poblacion"}, inplace= True)
    cities["Poblacion"] = cities["Poblacion"].astype(float)
    NFL["W-L%"] = NFL["W-L%"].astype(float)
    teams = cities["team"].str.extract("(?P<team1>^[\w]+)(?:[\s]*)(?P<team2>[\w]*)(?:[\s]*)(?P<team3>[\w]*$)")
    cities["team"] = teams["team1"]

    copia = pd.DataFrame()
    copia["Metropolitan area"] = cities["Metropolitan area"]
    copia["Poblacion"] = cities["Poblacion"]
    copia["team"] = teams["team2"]
    copia = copia[copia["team"] != ""]

    copia2 = pd.DataFrame()
    copia2["Metropolitan area"] = cities["Metropolitan area"]
    copia2["Poblacion"] = cities["Poblacion"]
    copia2["team"] = teams["team3"]
    copia2 = copia2[copia2["team"] != ""]

    cities = cities.set_index("team")
    copia = copia.set_index("team")
    copia2 = copia2.set_index("team")

    merge1 = pd.merge(NFL,cities,how="inner", left_index= True, right_index=True)
    merge2 = pd.merge(NFL,copia,how="inner", left_index= True, right_index=True)
    merge3 = pd.merge(NFL,copia2,how="inner", left_index= True, right_index=True)
    merge = pd.concat((merge1,merge2,merge3))
    
    
    merge=merge.groupby('Metropolitan area').agg({'W-L%': np.nanmean, 'Poblacion': np.nanmean})
    return merge

def nfl_correlation(): 
    merge = df_nfl()
    population_by_region =merge["Poblacion"] # pass in metropolitan area population from cities
    win_loss_by_region = merge["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [104]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def creator(sport):
    if sport =='NFL':
        return df_nfl()
    elif sport =='NBA':
        return df_nba()
    elif sport =='NHL':
        return df_nhl()
    elif sport =='MLB':
        return df_mlb()

def sports_team_performance():
    
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    print()
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in range(len(sports)):
            for j in range(len(sports)):
                if i !=j :
                    merge_temp = pd.merge(creator(sports[i]), creator(sports[j]), how='inner', left_index=True, right_index=True)
                    del merge_temp["Poblacion_x"]
                    del merge_temp["Poblacion_y"]
                    

                    
                    p_values.loc[sports[i], sports[j]] = stats.ttest_rel(merge_temp.iloc[:, [0]], merge_temp.iloc[:, [1]])[1]
    
    
    return p_values
sports_team_performance()

In [105]:
print(sports_team_performance())

          NFL       NBA       NHL       MLB
NFL       NaN  0.937509  0.030318  0.803459
NBA  0.937509       NaN  0.022386  0.949566
NHL  0.030318  0.022386       NaN  0.000703
MLB  0.803459  0.949566  0.000703       NaN
